In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LinearRegression, Lasso, LogisticRegressionCV, LogisticRegression
from sklearn.metrics import make_scorer, brier_score_loss
from sklearn.model_selection import cross_val_score

import sys
sys.path.append('../src/')
from ffcDataCleaning.CodeBook import getCodeDescription

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
gen_features = pd.read_csv('../output/imputing/.data/imputed.csv', index_col=0)
gen_features.head()
print gen_features.shape

In [ ]:
specific_features = pd.read_csv('../output/imputing/.data/jobSkill_features.csv', index_col=0)
specific_features.head()
specific_features['total'] = specific_features.sum(axis=1)
print specific_features.shape

In [ ]:
features = specific_features#pd.concat([specific_features,gen_features], axis = 1)
print features.shape
features.head()

In [ ]:
data_dir = '../.data'
fp_train = '{}/train.csv'.format(data_dir)
df_test = pd.read_csv(fp_train,index_col=0)

In [ ]:
Y = df_test.jobTraining.dropna()
print Y.shape
idx = Y.index

S = StandardScaler()
idx_features = features.index
FEATURES_TRAIN = S.fit_transform(features)
X = FEATURES_TRAIN[idx-1,:]
print X.shape

In [ ]:
b = 500
reg = LogisticRegressionCV()
reg.fit(X[:b],Y[:b])
print reg.C_
print reg.coef_

In [ ]:
Y_pred = reg.predict(X[b:])
Y_prob = reg.predict_proba(X[b:])
print Y_prob[:,0].shape
plt.hist(Y_prob[:,1])
print brier_score_loss(Y[b:], Y_prob[:,1])

In [ ]:
def one_term_brier(x,y):
    return brier_score_loss(x,y[:,1])
brier_scorer = make_scorer(one_term_brier,greater_is_better=False,needs_proba=True)

#### Best Score: .193
#### Baseline: .202

In [ ]:
scores = cross_val_score(LogisticRegression(),X,Y,cv=10,scoring=brier_scorer)#,n_jobs=-1)
print abs(scores)
print scores.mean()

## Final

In [ ]:
final_reg = LogisticRegressionCV(cv=10, n_jobs=-1)
final_reg.fit(X,Y)
Y_Final = final_reg.predict_proba(FEATURES_TRAIN)[:,1]

In [ ]:
Y_Final = pd.DataFrame(Y_Final, index = idx_features, columns=['jobTraining'])

In [ ]:
#sanity checks
print Y_Final.shape
print brier_score_loss(Y.values,Y_Final.loc[idx].values)
Y_Final.hist(normed=True)

In [ ]:
Y_Final.to_csv('../output/solutions/Solution1/jobTraining.csv')